In [27]:
import pandas as pd
from datetime import datetime, timedelta
from Indicators import resample, detect_fractal_v2, SuperTrend, atr

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [28]:
script = "ethusd"

df = pd.read_csv(f"{script}_merged.csv", parse_dates=["date"])

df1 = df[df["date"] >= datetime(2025, 6, 1)].copy(deep=True)

dfx = resample(df1, "15T")

d:\CryptoSniper\Backtest\Indicators.py:33: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.resample(timeframe, origin="start_day").agg(ohlcv_dict)
d:\CryptoSniper\Backtest\Indicators.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  timeframe_delta = pd.to_timedelta(timeframe)


In [29]:
dfx.head()

,date,open,high,low,close,volume
0,2025-06-01 00:00:00,2528.45,2529.06,2518.41,2520.50,2974.521632
1,2025-06-01 00:15:00,2520.49,2523.32,2515.46,2517.70,3443.769333
2,2025-06-01 00:30:00,2517.80,2521.96,2515.90,2521.96,1348.412897
3,2025-06-01 00:45:00,2521.96,2526.51,2521.30,2526.08,2406.654460
4,2025-06-01 01:00:00,2526.08,2529.20,2518.34,2528.40,2097.741586


In [30]:
# detect_fractal_v2(dfx, 2)
# fractal_top	fractal_bottom	fractal_time_top	fractal_time_bottom

In [31]:
SuperTrend(dfx, 10, 3.0)

array([0, 1, 1, ..., 1, 1, 1], shape=(3093,))

In [32]:
# apply indicators

dfx = detect_fractal_v2(dfx, 2)
dfx["supertrend"] = SuperTrend(dfx, 10, 3.0)
atr(dfx)
dfx.dropna(inplace=True, axis=0)


dfx["p_atr"] = dfx["ATR"].shift(1)
dfx["p_supertrend"] = dfx["supertrend"].shift(1)
dfx["n_open"] = dfx["open"].shift(-1)

In [33]:
dfx.head()

,date,open,high,low,close,volume,fractal_top,fractal_bottom,fractal_time_top,fractal_time_bottom,supertrend,high-low,high-PrevClose,low-PrevClose,TrueRange,ATR,p_atr,p_supertrend,n_open
13,2025-06-01 03:15:00,2501.70,2506.24,2492.84,2502.22,2568.229748,2513.88,2492.20,2025-06-01 02:45:00,2025-06-01 01:45:00,1,13.40,4.53,8.87,13.40,10.961429,NaN,NaN,2502.22
14,2025-06-01 03:30:00,2502.22,2511.69,2502.22,2510.02,1592.834657,2513.88,2492.20,2025-06-01 02:45:00,2025-06-01 01:45:00,1,9.47,9.47,0.00,9.47,10.854898,10.961429,1.0,2510.02
15,2025-06-01 03:45:00,2510.02,2518.75,2510.02,2513.23,2110.102471,2513.88,2492.84,2025-06-01 02:45:00,2025-06-01 03:15:00,1,8.73,8.73,0.00,8.73,10.703120,10.854898,1.0,2513.30
16,2025-06-01 04:00:00,2513.30,2516.07,2511.70,2512.91,1144.082207,2513.88,2492.84,2025-06-01 02:45:00,2025-06-01 03:15:00,1,4.37,2.84,1.53,4.37,10.250754,10.703120,1.0,2512.91
17,2025-06-01 04:15:00,2512.91,2516.62,2512.91,2516.00,363.008874,2518.75,2492.84,2025-06-01 03:45:00,2025-06-01 03:15:00,1,3.71,3.71,0.00,3.71,9.783557,10.250754,1.0,2516.04


In [34]:
buy_entries = (
    (dfx["close"] > dfx["fractal_top"])
    & (dfx["date"] > dfx["fractal_time_top"])
    & (dfx["p_supertrend"] == -1)
)

sell_entries = (
    (dfx["close"] < dfx["fractal_bottom"])
    & (dfx["date"] > dfx["fractal_time_bottom"])
    & (dfx["p_supertrend"] == 1)
)


dfx["Buy Entry Cond"] = buy_entries
dfx["Sell Entry Cond"] = sell_entries


dfx.head()

,date,open,high,low,close,volume,fractal_top,fractal_bottom,fractal_time_top,fractal_time_bottom,supertrend,high-low,high-PrevClose,low-PrevClose,TrueRange,ATR,p_atr,p_supertrend,n_open,Buy Entry Cond,Sell Entry Cond
13,2025-06-01 03:15:00,2501.70,2506.24,2492.84,2502.22,2568.229748,2513.88,2492.20,2025-06-01 02:45:00,2025-06-01 01:45:00,1,13.40,4.53,8.87,13.40,10.961429,NaN,NaN,2502.22,False,False
14,2025-06-01 03:30:00,2502.22,2511.69,2502.22,2510.02,1592.834657,2513.88,2492.20,2025-06-01 02:45:00,2025-06-01 01:45:00,1,9.47,9.47,0.00,9.47,10.854898,10.961429,1.0,2510.02,False,False
15,2025-06-01 03:45:00,2510.02,2518.75,2510.02,2513.23,2110.102471,2513.88,2492.84,2025-06-01 02:45:00,2025-06-01 03:15:00,1,8.73,8.73,0.00,8.73,10.703120,10.854898,1.0,2513.30,False,False
16,2025-06-01 04:00:00,2513.30,2516.07,2511.70,2512.91,1144.082207,2513.88,2492.84,2025-06-01 02:45:00,2025-06-01 03:15:00,1,4.37,2.84,1.53,4.37,10.250754,10.703120,1.0,2512.91,False,False
17,2025-06-01 04:15:00,2512.91,2516.62,2512.91,2516.00,363.008874,2518.75,2492.84,2025-06-01 03:45:00,2025-06-01 03:15:00,1,3.71,3.71,0.00,3.71,9.783557,10.250754,1.0,2516.04,False,False


In [35]:
# dfx.head(20)

In [36]:
main_df = dfx.drop_duplicates(
    subset=["fractal_top", "fractal_time_top", "Buy Entry Cond"], keep="first"
)
entry_df = main_df.drop_duplicates(
    subset=["fractal_bottom", "fractal_time_bottom", "Sell Entry Cond"], keep="first"
)

In [37]:
entry_df.head(10)

,date,open,high,low,close,volume,fractal_top,fractal_bottom,fractal_time_top,fractal_time_bottom,supertrend,high-low,high-PrevClose,low-PrevClose,TrueRange,ATR,p_atr,p_supertrend,n_open,Buy Entry Cond,Sell Entry Cond
13,2025-06-01 03:15:00,2501.70,2506.24,2492.84,2502.22,2568.229748,2513.88,2492.20,2025-06-01 02:45:00,2025-06-01 01:45:00,1,13.40,4.53,8.87,13.40,10.961429,NaN,NaN,2502.22,False,False
17,2025-06-01 04:15:00,2512.91,2516.62,2512.91,2516.00,363.008874,2518.75,2492.84,2025-06-01 03:45:00,2025-06-01 03:15:00,1,3.71,3.71,0.00,3.71,9.783557,10.250754,1.0,2516.04,False,False
38,2025-06-01 09:30:00,2494.35,2494.55,2482.55,2485.57,12915.834226,2510.96,2499.11,2025-06-01 09:00:00,2025-06-01 08:15:00,1,12.00,0.20,11.80,12.00,8.266230,7.979017,1.0,2485.58,False,True
43,2025-06-01 10:45:00,2490.49,2494.59,2486.92,2493.66,1415.124192,2496.93,2477.51,2025-06-01 10:15:00,2025-06-01 09:45:00,1,7.67,4.10,3.57,7.67,8.420842,8.478599,1.0,2493.66,False,False
49,2025-06-01 12:15:00,2498.50,2499.69,2489.87,2493.40,1229.496853,2502.90,2486.92,2025-06-01 11:45:00,2025-06-01 10:45:00,1,9.82,1.22,8.60,9.82,8.146009,8.017240,1.0,2493.35,False,False
58,2025-06-01 14:30:00,2512.71,2516.00,2509.61,2514.03,2793.229022,2502.90,2468.35,2025-06-01 11:45:00,2025-06-01 12:30:00,-1,6.39,3.30,3.09,6.39,9.321892,9.547422,-1.0,2514.04,True,False
68,2025-06-01 17:00:00,2539.08,2543.68,2527.96,2539.39,8421.355693,2548.26,2504.21,2025-06-01 16:30:00,2025-06-01 15:00:00,-1,15.72,4.61,11.11,15.72,11.093185,10.737276,-1.0,2539.39,False,False
72,2025-06-01 18:00:00,2539.30,2539.30,2524.24,2529.89,4308.192885,2547.18,2527.96,2025-06-01 17:30:00,2025-06-01 17:00:00,-1,15.06,0.02,15.08,15.08,10.632051,10.289901,-1.0,2529.90,False,False
78,2025-06-01 19:30:00,2518.88,2522.59,2517.15,2519.96,1220.797257,2527.08,2493.42,2025-06-01 19:00:00,2025-06-01 18:30:00,1,5.44,3.71,1.73,5.44,12.197354,12.717151,1.0,2520.00,False,False
95,2025-06-01 23:45:00,2536.12,2539.60,2533.19,2539.21,1904.487286,2545.55,2532.67,2025-06-01 23:15:00,2025-06-01 22:45:00,-1,6.41,3.49,2.92,6.41,9.110472,9.318201,-1.0,2539.20,False,False


In [38]:
dfx["date"] = pd.to_datetime(dfx["date"], format="%Y-%m-%d %H:%M:%S")

In [39]:
entry_df.dropna(inplace=True, axis=0)
entry_df.reset_index(inplace=True)

C:\Users\vipin\AppData\Local\Temp\ipykernel_13212\1342345424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry_df.dropna(inplace=True, axis=0)


In [40]:
p = entry_df.iloc[0]["date"] - timedelta(days=1)
p

Timestamp('2025-05-31 04:15:00')

In [41]:
dfx.reset_index(inplace=True)

In [42]:
dfx.at[0, "date"]

Timestamp('2025-06-01 03:15:00')

In [43]:
## flags
prev_fractal = 0
prev_exit_time = dfx.at[0, "date"]
EntryPrice = 0
entry = False
side = ""
final_sheet = []


for i in range(len(entry_df) - 2):

    EntryTime = entry_df.at[i, "date"]

    if entry_df.at[i, "date"] < prev_exit_time:
        continue

    FractalTop = entry_df.at[i, "fractal_top"]
    FractalBottom = entry_df.at[i, "fractal_bottom"]
    FractalTopTime = entry_df.at[i, "fractal_time_top"]
    FractalBottomTime = entry_df.at[i, "fractal_time_bottom"]
    atr = entry_df.at[i, "p_atr"]
    spt = entry_df.at[i, "p_supertrend"]

    if entry_df.at[i, "Buy Entry Cond"] and FractalTopTime >= prev_exit_time:
        entry = True
        EntryPrice = entry_df.at[i + 1, "open"]
        side = "buy"
        prev_fractal = FractalTop
        prev_exit_time = entry_df["date"][i]

    if entry_df.at[i, "Sell Entry Cond"] and FractalBottomTime >= prev_exit_time:
        entry = True
        EntryPrice = entry_df.at[i + 1, "open"]
        side = "sell"
        prev_fractal = FractalBottom
        prev_exit_time = entry_df["date"][i]

    if entry and side == "buy":
        # print("Buy trade")
        sl = EntryPrice - (1 * atr)
        tg = EntryPrice + (2 * atr)
        sl_flt = dfx[(dfx["date"] > EntryTime) & (dfx["low"] < sl)]
        tg_flt = dfx[(dfx["date"] > EntryTime) & (dfx["high"] > tg)]

        if not sl_flt.empty and not tg_flt.empty:
            # check who is first
            first_sl = sl_flt.iloc[0]["date"]
            first_tg = tg_flt.iloc[0]["date"]

            if first_sl < first_tg:
                ExitPrice = sl
                ExitTime = first_sl
                ExitType = "SL"
            else:
                ExitPrice = tg
                ExitTime = first_tg
                ExitType = "TG"

        if sl_flt.empty and not tg_flt.empty:
            ExitPrice = tg
            ExitTime = tg_flt.iloc[0]["date"]
            ExitType = "TG"

        if tg_flt.empty and not sl_flt.empty:
            ExitPrice = sl
            ExitTime = sl_flt.iloc[0]["date"]
            ExitType = "SL"

        prev_exit_time = ExitTime
        doc = {
            "EntryTime": EntryTime,
            "EntryPrice": EntryPrice,
            "Side": side,
            "FractalPrice": FractalTop,
            "FractalTime": FractalTopTime,
            "SuperTrend": spt,
            "Atr": atr,
            "StopLoss": sl,
            "Target": tg,
            "ExitType": ExitType,
            "ExitPrice": ExitPrice,
            "ExitTime": ExitTime,
        }

        final_sheet.append(doc)

    if entry and side == "sell":
        # print("Sell trade")
        sl = EntryPrice + (1 * atr)  # SL above entry for sell
        tg = EntryPrice - (2 * atr)  # TG below entry for sell
        sl_flt = dfx[(dfx["date"] > EntryTime) & (dfx["high"] > sl)]
        tg_flt = dfx[(dfx["date"] > EntryTime) & (dfx["low"] < tg)]

        if not sl_flt.empty and not tg_flt.empty:
            # check who is first
            first_sl = sl_flt.iloc[0]["date"]
            first_tg = tg_flt.iloc[0]["date"]

            if first_sl < first_tg:
                ExitPrice = sl
                ExitTime = first_sl
                ExitType = "SL"
            else:
                ExitPrice = tg
                ExitTime = first_tg
                ExitType = "TG"

        if sl_flt.empty and not tg_flt.empty:
            ExitPrice = tg
            ExitTime = tg_flt.iloc[0]["date"]
            ExitType = "TG"

        if tg_flt.empty and not sl_flt.empty:
            ExitPrice = sl
            ExitTime = sl_flt.iloc[0]["date"]
            ExitType = "SL"

        prev_exit_time = ExitTime
        doc = {
            "EntryTime": EntryTime,
            "EntryPrice": EntryPrice,
            "Side": side,
            "FractalPrice": FractalTop,
            "FractalTime": FractalTopTime,
            "SuperTrend": spt,
            "Atr": atr,
            "StopLoss": sl,
            "Target": tg,
            "ExitType": ExitType,
            "ExitPrice": ExitPrice,
            "ExitTime": ExitTime,
        }

        final_sheet.append(doc)

In [44]:
FractalTop, FractalBottom, FractalTopTime, FractalBottomTime, atr, entry, side

(np.float64(2590.39),
 np.float64(2567.61),
 Timestamp('2025-07-02 17:15:00'),
 Timestamp('2025-07-02 19:00:00'),
 np.float64(13.231517091414405),
 True,
 'buy')

In [45]:
sheet = pd.DataFrame(final_sheet)
sheet.head()

,EntryTime,EntryPrice,Side,FractalPrice,FractalTime,SuperTrend,Atr,StopLoss,Target,ExitType,ExitPrice,ExitTime
0,2025-06-01 09:30:00,2490.49,sell,2510.96,2025-06-01 09:00:00,1.0,7.979017,2498.469017,2474.531965,SL,2498.469017,2025-06-01 11:15:00
1,2025-06-01 12:15:00,2490.49,sell,2502.90,2025-06-01 11:45:00,1.0,8.017240,2498.507240,2474.455519,TG,2474.455519,2025-06-01 12:30:00
2,2025-06-01 14:30:00,2490.49,sell,2502.90,2025-06-01 11:45:00,-1.0,9.547422,2500.037422,2471.395155,SL,2500.037422,2025-06-01 14:45:00
3,2025-06-01 17:00:00,2490.49,sell,2548.26,2025-06-01 16:30:00,-1.0,10.737276,2501.227276,2469.015447,SL,2501.227276,2025-06-01 17:15:00
4,2025-06-01 18:00:00,2490.49,sell,2547.18,2025-06-01 17:30:00,-1.0,10.289901,2500.779901,2469.910198,SL,2500.779901,2025-06-01 18:15:00


In [46]:
import numpy as np

sheet["pnl"] = np.where(
    sheet["Side"] == "buy",
    sheet["ExitPrice"] - sheet["EntryPrice"],
    sheet["EntryPrice"] - sheet["ExitPrice"],
)

In [47]:
sheet.tail(12)

,EntryTime,EntryPrice,Side,FractalPrice,FractalTime,SuperTrend,Atr,StopLoss,Target,ExitType,ExitPrice,ExitTime,pnl
313,2025-07-01 10:30:00,2483.27,buy,2461.38,2025-07-01 10:00:00,1.0,7.061366,2476.208634,2497.392731,SL,2476.208634,2025-07-01 10:45:00,-7.061366
314,2025-07-01 14:45:00,2422.73,sell,2456.24,2025-07-01 14:15:00,1.0,11.689062,2434.419062,2399.351876,TG,2399.351876,2025-07-01 18:45:00,23.378124
315,2025-07-01 23:00:00,2397.63,sell,2412.42,2025-07-01 22:30:00,1.0,9.728595,2407.358595,2378.172809,SL,2407.358595,2025-07-02 00:15:00,-9.728595
316,2025-07-02 01:00:00,2397.63,sell,2409.11,2025-07-02 00:30:00,1.0,8.600136,2406.230136,2380.429727,TG,2380.429727,2025-07-02 01:15:00,17.200273
317,2025-07-02 02:30:00,2397.63,sell,2417.28,2025-07-02 02:00:00,1.0,9.480106,2407.110106,2378.669787,SL,2407.110106,2025-07-02 02:45:00,-9.480106
318,2025-07-02 03:15:00,2397.63,sell,2417.28,2025-07-02 02:00:00,-1.0,8.921893,2406.551893,2379.786215,SL,2406.551893,2025-07-02 03:30:00,-8.921893
319,2025-07-02 04:30:00,2397.63,sell,2431.61,2025-07-02 03:15:00,-1.0,8.180766,2405.810766,2381.268468,SL,2405.810766,2025-07-02 04:45:00,-8.180766
320,2025-07-02 06:45:00,2450.81,buy,2448.42,2025-07-02 06:00:00,-1.0,7.807179,2443.002821,2466.424358,SL,2443.002821,2025-07-02 07:00:00,-7.807179
321,2025-07-02 09:45:00,2450.81,buy,2452.70,2025-07-02 09:15:00,-1.0,6.737736,2444.072264,2464.285472,SL,2444.072264,2025-07-02 10:00:00,-6.737736
322,2025-07-02 10:45:00,2450.81,buy,2452.70,2025-07-02 09:15:00,-1.0,6.144591,2444.665409,2463.099181,SL,2444.665409,2025-07-02 11:45:00,-6.144591


In [ ]:
sheet.to_excel("SPT_FRA_BACKTEST.xlsx", index=False)

In [49]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------

In [ ]:
dfx.tail()

NameError: name 'dfx' is not defined

In [ ]:
from datetime import datetime
import time

tm = [0, 15, 30, 45]

while True:
    dt_m = datetime.now().minute
    if dt_m in tm:
        print("Time", dt_m)
        time.sleep(60)
        break

Time 30
